---

# Sequence to Sequence Models (COSC 6336)
### Solution of HW3

<br>
<div style="text-align: right; font-size: large;"><i>authored by Gustavo Aguilar</i></div>
<div style="text-align: right; font-size: small;"><i>March 19, 2020</i></div>

---


# Dataset

In [1]:
import random
import torch
torch.manual_seed(1)
random.seed(1)

class Vocab:
    def __init__(self, vocab_list):
        self.itos = vocab_list
        self.stoi = {d:i for i, d in enumerate(self.itos)}
        
    def __len__(self):
        return len(self.itos)  
    
def sorting_letters_dataset(size):
    dataset = []
    for _ in range(size):
        x = []
        for _ in range(random.randint(3, 8)):
            letter = chr(random.randint(97, 122))
            repeat = [letter] * random.randint(1, 3)
            x.extend(repeat)
        y = sorted(set(x))
        dataset.append((x, y))
    return zip(*dataset)

src_vocab = Vocab(['<pad>'] + [chr(i+97) for i in range(26)])
tgt_vocab = Vocab(['<pad>'] + [chr(i+97) for i in range(26)] + ['<start>', '<stop>'] )

train_inp, train_out = sorting_letters_dataset(20_000)
valid_inp, valid_out = sorting_letters_dataset(5_000)

print("Encoder Vocab:", src_vocab.itos)
print("Decoder Vocab:", tgt_vocab.itos)

Encoder Vocab: ['<pad>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Decoder Vocab: ['<pad>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '<start>', '<stop>']


# Encodings: use the mapped indexes

In [2]:
def map_elems(elems, mapper):
    return [mapper[elem] for elem in elems]

def map_many_elems(many_elems, mapper):
    return [map_elems(elems, mapper) for elems in many_elems]

def add_start_stop(start, stop, many_elems):
    return [[start] + elems + [stop] for elems in many_elems]

train_x = map_many_elems(train_inp, src_vocab.stoi)
train_y = map_many_elems(train_out, tgt_vocab.stoi)

train_y = add_start_stop(tgt_vocab.stoi['<start>'], tgt_vocab.stoi['<stop>'], train_y)

In [3]:
print(train_x[0], train_inp[0])
print(train_y[0], train_out[0])

[19, 9, 16, 16, 16, 16, 16] ['s', 'i', 'p', 'p', 'p', 'p', 'p']
[27, 9, 16, 19, 28] ['i', 'p', 's']


# Seq2Seq Model

In [4]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, lstm_dim=256, lstm_layers=2, dropout=0.5):
        super(Encoder, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, lstm_dim, lstm_layers, dropout=dropout)
        self.drop = nn.Dropout(dropout)
    
    def forward(self, inputs):
        embs = self.drop(self.emb(inputs))
        outs, (hidden, cell) = self.lstm(embs)
        return outs, (hidden, cell)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, lstm_dim=256, lstm_layers=2, dropout=0.5):
        super(Decoder, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, lstm_dim, lstm_layers, dropout=dropout)
        self.clf = nn.Linear(lstm_dim, vocab_size)
        self.drop = nn.Dropout(dropout)
        
    def forward(self, input_i, state):
        input_i = input_i.view(1, -1)          # (1, batch)
        emb = self.drop(self.emb(input_i))     # (1, batch, emb)
        output, state = self.lstm(emb, state)  # (1, batch, hid), ((layers, batch, hid), ...)
        score = self.clf(output.squeeze(0)) # (batch, vocab)
        return score, state

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, inputs, targets):
        scores = []
        _, state = self.encoder(inputs)
        
        for i in range(1, targets.shape[0]):
            input_i = targets[i-1]
            score, state = self.decoder(input_i, state)
            scores.append(score.unsqueeze(0))
        
        scores = torch.cat(scores, dim=0)
        return scores
    
    def predict(self, sample, start, stop, maxlen):
        preds = []    
        _, state = self.encoder(sample.view(-1, 1))

        token = start
        for i in range(maxlen):
            score, state = self.decoder(token, state)
            token = score.argmax(dim=1)
            if token == stop:
                break
            preds.append(token.item())
        return preds
    
seq2seq = Seq2Seq(
    encoder=Encoder(len(src_vocab), emb_dim=64, lstm_dim=128, lstm_layers=1), 
    decoder=Decoder(len(tgt_vocab), emb_dim=64, lstm_dim=128, lstm_layers=1)
)
seq2seq

/home/gaguil20/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Seq2Seq(
  (encoder): Encoder(
    (emb): Embedding(27, 64)
    (lstm): LSTM(64, 128, dropout=0.5)
    (drop): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (emb): Embedding(29, 64)
    (lstm): LSTM(64, 128, dropout=0.5)
    (clf): Linear(in_features=128, out_features=29, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
  )
)

# Seq2Seq+Attention Model

In [5]:
class Attention(nn.Module):
    def __init__(self, input_dim, attn_dim):
        super(Attention, self).__init__()
        self.W = nn.Linear(input_dim, attn_dim)
        self.v = nn.Linear(attn_dim, 1, bias=False)
        
    def forward(self, dec_hidden, enc_outs):
        # enc_outs -> (seqlen, batch, dim)
        # dec_hidden -> (lstm_layers, batch, hid)
        
        seqlen = enc_outs.size(0)
        repeat_h = dec_hidden.repeat(seqlen, 1, 1)
        concat_h = torch.cat((enc_outs, repeat_h), dim=2) 
        
        scores = self.v(torch.tanh(self.W(concat_h))) # (seqlen, batch, 1)
        probs = torch.softmax(scores, dim=0)
        
        weighted = enc_outs * probs # (seqlen, batch, hidden)
        
        context = torch.sum(weighted, dim=0, keepdim=True) # (1, batch, hidden)
        combined = torch.cat((dec_hidden, context), dim=2)  # (1, batch, hidden*2)
        
        return combined

class AttnDecoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, lstm_dim=256, lstm_layers=2, attn_size=100, dropout=0.5):
        super(AttnDecoder, self).__init__()
        self.lstm_dim = lstm_dim
        self.lstm_layers = lstm_layers
        
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, lstm_dim, lstm_layers, dropout=dropout)
        self.attn = Attention(lstm_dim * 2, attn_size)
        self.clf = nn.Linear(lstm_dim * 2, vocab_size)
        self.drop = nn.Dropout(dropout)
        
    def init_state(self, batch_size, device):
        h_0 = torch.zeros(self.lstm_layers, batch_size, self.lstm_dim).to(device)
        c_0 = torch.zeros(self.lstm_layers, batch_size, self.lstm_dim).to(device)
        return h_0, c_0
    
    def forward(self, input_i, state, enc_outs):
        # enc_outs -> (seqlen, batch, dim)
        
        input_i = input_i.view(1, -1)   # (1, batch)
        batch_size = input_i.size(1)
        
        emb = self.emb(input_i)         # (1, batch, emb)
        emb = self.drop(emb)
        
        output, state = self.lstm(emb, state)  # (1, batch, hidden), ((layers, batch, hidden), ...)
        combined = self.attn(output, enc_outs) # (1, batch, hidden)
        score = self.clf(combined)          # (1, batch, vocab)
        
        return score, state
    
class Seq2SeqAttn(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqAttn, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, inputs, targets):
        scores = []
        outs, _ = self.encoder(inputs)
        state = self.decoder.init_state(targets.shape[1], inputs.device)
        for i in range(1, targets.shape[0]):
            input_i = targets[i-1]
            score, state = self.decoder(input_i, state, outs) 
            scores.append(score) # (1, batch, vocab)
        
        scores = torch.cat(scores, dim=0)
        return scores
    
    def predict(self, sample, start, stop, maxlen):
        preds = []    
        outs, _ = self.encoder(sample.view(-1, 1))
        state = self.decoder.init_state(1, sample.device)
        token = start
        for i in range(maxlen):
            score, state = self.decoder(token, state, outs)
            token = score.argmax(dim=2)
            if token == stop:
                break
            preds.append(token.item())
        return preds
    

seq2seq_attn = Seq2SeqAttn(
    encoder=Encoder(len(src_vocab), emb_dim=64, lstm_dim=128, lstm_layers=1, dropout=0.5), 
    decoder=AttnDecoder(len(tgt_vocab), emb_dim=64, lstm_dim=128, lstm_layers=1, attn_size=100, dropout=0.5)
)
seq2seq_attn

Seq2SeqAttn(
  (encoder): Encoder(
    (emb): Embedding(27, 64)
    (lstm): LSTM(64, 128, dropout=0.5)
    (drop): Dropout(p=0.5, inplace=False)
  )
  (decoder): AttnDecoder(
    (emb): Embedding(29, 64)
    (lstm): LSTM(64, 128, dropout=0.5)
    (attn): Attention(
      (W): Linear(in_features=256, out_features=100, bias=True)
      (v): Linear(in_features=100, out_features=1, bias=False)
    )
    (clf): Linear(in_features=256, out_features=29, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
  )
)

# Training

In [6]:
def create_batch(x, pad_ix):
    batch_size = len(x)
    maxlen = max([len(xi) for xi in x])
    batch = [xi + [pad_ix] * (maxlen - len(xi)) for xi in x]
    batch = torch.tensor(batch)
    return batch.transpose(0, 1).contiguous()

input_batch = create_batch(train_x[:10], pad_ix=src_vocab.stoi['<pad>'])
target_batch = create_batch(train_y[:10], pad_ix=tgt_vocab.stoi['<pad>'])

print(input_batch, input_batch.shape)
print(target_batch, target_batch.shape)

tensor([[19, 26, 24, 13,  8,  4, 19, 22, 25,  1],
        [ 9,  4, 19, 13,  8, 21, 19, 12, 17,  1],
        [16,  4, 11, 13, 16, 21, 17, 12, 17,  2],
        [16,  1,  1,  7, 16, 21, 17, 12, 12,  2],
        [16,  1, 21,  7, 16, 10, 19, 23, 12, 23],
        [16, 14, 21, 24,  8, 24, 16, 23,  0, 23],
        [16, 14, 21,  0,  8, 24, 24, 23,  0, 23],
        [ 0, 14,  0,  0,  8, 24, 24, 24,  0, 19],
        [ 0, 25,  0,  0,  8, 24,  0, 24,  0, 19],
        [ 0, 23,  0,  0,  8, 24,  0,  3,  0, 19],
        [ 0, 23,  0,  0,  8, 17,  0,  3,  0, 13],
        [ 0,  0,  0,  0,  8, 17,  0, 22,  0, 13],
        [ 0,  0,  0,  0, 10, 17,  0, 22,  0, 13],
        [ 0,  0,  0,  0, 14, 17,  0, 22,  0,  6],
        [ 0,  0,  0,  0, 14, 17,  0,  0,  0, 17],
        [ 0,  0,  0,  0, 14,  7,  0,  0,  0,  1],
        [ 0,  0,  0,  0,  0,  7,  0,  0,  0,  0]]) torch.Size([17, 10])
tensor([[27, 27, 27, 27, 27, 27, 27, 27, 27, 27],
        [ 9,  1,  1,  7,  8,  4, 16,  3, 12,  1],
        [16,  4, 11, 13, 10,

In [7]:
import torch.optim as optim

def shuffle(x, y):
    pack = list(zip(x, y))
    random.shuffle(pack)
    return zip(*pack)

def train(model, inputs, targets, optimizer, criterion, config):
    model.to(config['device'])
    for epoch in range(1, config['epochs']+1):
        epoch_loss = 0
        
        inputs, targets = shuffle(inputs, targets)
        model.train()
        
        n_batches = len(inputs) // config['batch_size']
        for batch_i in range(n_batches):
            optimizer.zero_grad()
            model.zero_grad()
            
            start_ix = config['batch_size'] * batch_i
            end_ix = config['batch_size'] * (batch_i + 1)
            
            x_batch = create_batch(inputs[start_ix: end_ix], pad_ix=0).to(config['device'])
            y_batch = create_batch(targets[start_ix: end_ix], pad_ix=0).to(config['device'])
        
            scores = model(x_batch, y_batch)
            loss = criterion(scores.view(-1, scores.shape[-1]), y_batch[1:].view(-1))
            
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        print(f"Epoch {epoch} - Loss: {epoch_loss / n_batches:.6f}")

### Train the seq2seq model

In [8]:
optimizer = optim.SGD(seq2seq.parameters(), lr=0.001, momentum=0.99)
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=tgt_vocab.stoi['<pad>'])
      
config = {
    'device': torch.device('cuda:1' if torch.cuda.is_available() else 'cpu'),
    'epochs': 100,
    'batch_size': 50
}
train(seq2seq, train_x, train_y, optimizer, criterion, config)

Epoch 1 - Loss: 3.006575
Epoch 2 - Loss: 2.489694
Epoch 3 - Loss: 2.324506
Epoch 4 - Loss: 2.254594
Epoch 5 - Loss: 2.210481
Epoch 6 - Loss: 2.166394
Epoch 7 - Loss: 2.117718
Epoch 8 - Loss: 2.049933
Epoch 9 - Loss: 1.970129
Epoch 10 - Loss: 1.874371
Epoch 11 - Loss: 1.754994
Epoch 12 - Loss: 1.596474
Epoch 13 - Loss: 1.452497
Epoch 14 - Loss: 1.322944
Epoch 15 - Loss: 1.198761
Epoch 16 - Loss: 1.083497
Epoch 17 - Loss: 0.973898
Epoch 18 - Loss: 0.869569
Epoch 19 - Loss: 0.785726
Epoch 20 - Loss: 0.711441
Epoch 21 - Loss: 0.642916
Epoch 22 - Loss: 0.584787
Epoch 23 - Loss: 0.530618
Epoch 24 - Loss: 0.487720
Epoch 25 - Loss: 0.445964
Epoch 26 - Loss: 0.410633
Epoch 27 - Loss: 0.377951
Epoch 28 - Loss: 0.347941
Epoch 29 - Loss: 0.324617
Epoch 30 - Loss: 0.301084
Epoch 31 - Loss: 0.279771
Epoch 32 - Loss: 0.261731
Epoch 33 - Loss: 0.244669
Epoch 34 - Loss: 0.225305
Epoch 35 - Loss: 0.211502
Epoch 36 - Loss: 0.198190
Epoch 37 - Loss: 0.186353
Epoch 38 - Loss: 0.172565
Epoch 39 - Loss: 0.16

In [9]:
torch.save({'encoder': seq2seq.encoder.state_dict(),
            'decoder': seq2seq.decoder.state_dict()}, 'seq2seq.pt')

# Evaluate

Prepare the validation set

In [10]:
valid_x = map_many_elems(valid_inp, src_vocab.stoi)
valid_y = map_many_elems(valid_out, tgt_vocab.stoi)

valid_y = add_start_stop(tgt_vocab.stoi['<start>'], tgt_vocab.stoi['<stop>'], valid_y)

Prepare the test set

In [11]:
import pandas as pd

df = pd.read_csv('test.txt', delimiter='\t', header=None, usecols=[0,1])
test_inp = df[0].tolist()
test_out = df[1].tolist()

test_x = map_many_elems(test_inp, src_vocab.stoi)
test_y = map_many_elems(test_out, tgt_vocab.stoi)

test_y = add_start_stop(tgt_vocab.stoi['<start>'], tgt_vocab.stoi['<stop>'], test_y)

Define the evaluation function

In [12]:
from sklearn.metrics import accuracy_score

def inspect_and_eval(model, inputs, targets, src_vocab, tgt_vocab, device):
    model.eval()
    
    predictions = []
    groundtruth = []
    
    start = torch.tensor([tgt_vocab.stoi['<start>']]).to(device)
    stop = torch.tensor([tgt_vocab.stoi['<stop>']]).to(device)
    
    for i in range(len(inputs)):
        x = torch.tensor(inputs[i]).to(device)
        preds = model.predict(x, start, stop, maxlen=len(targets[i]))
        
        input_str  = ''.join([src_vocab.itos[ix] for ix in inputs[i]])
        output_str = ''.join([tgt_vocab.itos[ix] for ix in targets[i][1:-1]])
        prediction = ''.join([tgt_vocab.itos[ix] for ix in preds])
        
        predictions.append(prediction)
        groundtruth.append(output_str)
        
        if i < 20: 
            print(f"{input_str} --> {output_str} --> {prediction}")
            
    print("\nAccuracy:", accuracy_score(groundtruth, predictions))

## Results of the seq2seq model

In [13]:
inspect_and_eval(seq2seq, valid_x, valid_y, src_vocab, tgt_vocab, config['device'])

sskkkwwwvkkcc --> cksvw --> cksvw
rrqqqppyyyyzqqggg --> gpqryz --> gpqryz
traaa --> art --> art
yylllfflwvvw --> flvwy --> flvwy
uurryyeeeppp --> epruy --> epruy
apppfffnaaannnhh --> afhnp --> afhnp
wwppllzllsjxx --> jlpswxz --> jlpswxz
plloovaahhh --> ahlopv --> ahlopv
oooiiuwwwjjjnn --> ijnouw --> ijnouw
immmbgg --> bgim --> bgim
ppmmhhhdduu --> dhmpu --> dhmpu
iithvv --> hitv --> hitv
aaxxwyyyxxxccce --> acewxy --> acewxy
bbbcuuuii --> bciu --> bciu
ddffyyygooovbbll --> bdfglovy --> bdfglovy
ttxppiiiyyyce --> ceiptxy --> ceiptxy
slllyy --> lsy --> lsy
nnntttr --> nrt --> nrt
hhbwbbnnppfffmmm --> bfhmnpw --> bfhmnpw
sljjffk --> fjkls --> fjkls

Accuracy: 0.9462


In [14]:
inspect_and_eval(seq2seq, test_x, test_y, src_vocab, tgt_vocab, config['device'])

wwwttttjjjhhddmmmmqqffmmmdddcffu --> cdfhjmqtuw --> cdfhjmqtuw
cccrrrvvuhwwwwwrmmm --> chmruvw --> chmruvw
rzzzzxxxxxlyyyyykkmmmmaaaaaeeeeaahhhwwwwwiiii --> aehiklmrwxyz --> aehiklmwyz
rrrzzpppmmmhhhh --> hmprz --> hmprz
iiiidggggdddddmmjoonnnnnjjjy --> dgijmnoy --> dgijmnoy
ffflllllrrrsssyybttttaa --> abflrsty --> abflrsty
oooooooojjjjjubbbbbyyy --> bjouy --> bjouy
cnnrrrrrlllllzqnncccrrrrrxxsssssqqqqq --> clnqrsxz --> clnqrsxz
ppppteeeehhzzzxx --> ehptxz --> ehptxz
fvvvvvvvggggg --> fgv --> fgv
iiijjkkkrrrr --> ijkr --> ijkr
uuuuujjpppppqqqqqgggggeeeeeeaasss --> aegjpqsu --> aegjpqsu
llyyccccc --> cly --> cly
cceewppperrrqqmmf --> cefmpqrw --> cefmpqrw
mmiiiayyyyyzzzlllllmmm --> ailmyz --> ailmyz
qqqfffkklllldddddggguuukkkkkcppp --> cdfgklpqu --> cdfgklpqu
yyyyyqwwwwwazyyyyyppppz --> apqwyz --> apqwyz
ffffbbbbfffffzzzzzrriiccccchrrrrr --> bcfhirz --> bcfhirz
ppwwwwwwwqqqlllllmmmmcccccccccooouuu --> clmopquw --> clmopquw
gggrrryyyyyvvvvccc --> cgrvy --> cgrvy

Accuracy: 0.9006


## Results of the seq2seq+attention model

### Train the seq2seq+attention model

In [15]:
optimizer = optim.SGD(seq2seq_attn.parameters(), lr=0.001, momentum=0.99)

train(seq2seq_attn, train_x, train_y, optimizer, criterion, config)

Epoch 1 - Loss: 3.025384
Epoch 2 - Loss: 2.419975
Epoch 3 - Loss: 2.092274
Epoch 4 - Loss: 1.798615
Epoch 5 - Loss: 1.527555
Epoch 6 - Loss: 1.290155
Epoch 7 - Loss: 1.074695
Epoch 8 - Loss: 0.882867
Epoch 9 - Loss: 0.698329
Epoch 10 - Loss: 0.546922
Epoch 11 - Loss: 0.425365
Epoch 12 - Loss: 0.330464
Epoch 13 - Loss: 0.262080
Epoch 14 - Loss: 0.208691
Epoch 15 - Loss: 0.173486
Epoch 16 - Loss: 0.151353
Epoch 17 - Loss: 0.127625
Epoch 18 - Loss: 0.115331
Epoch 19 - Loss: 0.102455
Epoch 20 - Loss: 0.093636
Epoch 21 - Loss: 0.083018
Epoch 22 - Loss: 0.075621
Epoch 23 - Loss: 0.071402
Epoch 24 - Loss: 0.068432
Epoch 25 - Loss: 0.061416
Epoch 26 - Loss: 0.058389
Epoch 27 - Loss: 0.054462
Epoch 28 - Loss: 0.053237
Epoch 29 - Loss: 0.049783
Epoch 30 - Loss: 0.047174
Epoch 31 - Loss: 0.043020
Epoch 32 - Loss: 0.043047
Epoch 33 - Loss: 0.041037
Epoch 34 - Loss: 0.038855
Epoch 35 - Loss: 0.038586
Epoch 36 - Loss: 0.036598
Epoch 37 - Loss: 0.034978
Epoch 38 - Loss: 0.032683
Epoch 39 - Loss: 0.03

In [16]:
torch.save({'encoder': seq2seq_attn.encoder.state_dict(),
            'decoder': seq2seq_attn.decoder.state_dict()}, 'seq2seq_attn.pt')

In [17]:
inspect_and_eval(seq2seq_attn, valid_x, valid_y, src_vocab, tgt_vocab, config['device'])

sskkkwwwvkkcc --> cksvw --> cksvw
rrqqqppyyyyzqqggg --> gpqryz --> gpqryz
traaa --> art --> art
yylllfflwvvw --> flvwy --> flvwy
uurryyeeeppp --> epruy --> epruy
apppfffnaaannnhh --> afhnp --> afhnp
wwppllzllsjxx --> jlpswxz --> jlpswxz
plloovaahhh --> ahlopv --> ahlopv
oooiiuwwwjjjnn --> ijnouw --> ijnouw
immmbgg --> bgim --> bgim
ppmmhhhdduu --> dhmpu --> dhmpu
iithvv --> hitv --> hitv
aaxxwyyyxxxccce --> acewxy --> acewxy
bbbcuuuii --> bciu --> bciu
ddffyyygooovbbll --> bdfglovy --> bdfglovy
ttxppiiiyyyce --> ceiptxy --> ceiptxy
slllyy --> lsy --> lsy
nnntttr --> nrt --> nrt
hhbwbbnnppfffmmm --> bfhmnpw --> bfhmnpw
sljjffk --> fjkls --> fjkls

Accuracy: 0.9542


In [18]:
inspect_and_eval(seq2seq_attn, test_x, test_y, src_vocab, tgt_vocab, config['device'])

wwwttttjjjhhddmmmmqqffmmmdddcffu --> cdfhjmqtuw --> cdfhjmqtuw
cccrrrvvuhwwwwwrmmm --> chmruvw --> chmruvw
rzzzzxxxxxlyyyyykkmmmmaaaaaeeeeaahhhwwwwwiiii --> aehiklmrwxyz --> aehiklmrwxyz
rrrzzpppmmmhhhh --> hmprz --> hmprz
iiiidggggdddddmmjoonnnnnjjjy --> dgijmnoy --> dgijmnoy
ffflllllrrrsssyybttttaa --> abflrsty --> abflrsty
oooooooojjjjjubbbbbyyy --> bjouy --> bjouy
cnnrrrrrlllllzqnncccrrrrrxxsssssqqqqq --> clnqrsxz --> clnqrsxz
ppppteeeehhzzzxx --> ehptxz --> ehptxz
fvvvvvvvggggg --> fgv --> fgvv
iiijjkkkrrrr --> ijkr --> ijkr
uuuuujjpppppqqqqqgggggeeeeeeaasss --> aegjpqsu --> aegjpqsu
llyyccccc --> cly --> cly
cceewppperrrqqmmf --> cefmpqrw --> cefmpqrw
mmiiiayyyyyzzzlllllmmm --> ailmyz --> ailmyz
qqqfffkklllldddddggguuukkkkkcppp --> cdfgklpqu --> cdfgklpqu
yyyyyqwwwwwazyyyyyppppz --> apqwyz --> apqwyz
ffffbbbbfffffzzzzzrriiccccchrrrrr --> bcfhirz --> bcfhirz
ppwwwwwwwqqqlllllmmmmcccccccccooouuu --> clmopquw --> clmopquw
gggrrryyyyyvvvvccc --> cgrvy --> cgrvy

Accuracy: 0.9778
